# Building a simple neural-network with Keras



In [ ]:
!pip install -q modelify

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = (10,10) # Make the figures a bit bigger

from keras.datasets import mnist
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation
from keras.utils import np_utils
import tensorflow as tf

### Load training data

In [ ]:
from keras.datasets.mnist import load_data

# load the data - it returns 2 tuples of digits & labels - one for
# the train set & the other for the test set
(train_digits, train_labels), (test_digits, test_labels) = load_data()

# display 14 random images from the training set
import numpy as np
np.random.seed(123)

rand_14 = np.random.randint(0, train_digits.shape[0],14)
sample_digits = train_digits[rand_14]
sample_labels = train_labels[rand_14]
# code to view the images
num_rows, num_cols = 2, 7
f, ax = plt.subplots(num_rows, num_cols, figsize=(12,5),
                     gridspec_kw={'wspace':0.03, 'hspace':0.01}, 
                     squeeze=True)

for r in range(num_rows):
    for c in range(num_cols):
        image_index = r * 7 + c
        ax[r,c].axis("off")
        ax[r,c].imshow(sample_digits[image_index], cmap='gray')
        ax[r,c].set_title('No. %d' % sample_labels[image_index])
plt.show()
plt.close()

### Preprocess


In [ ]:
# some variables...
image_height = train_digits.shape[1]  
image_width = train_digits.shape[2]
num_channels = 1  # we have grayscale images
# NOTE: image_height == image_width == 28

# re-shape the images data
train_data = np.reshape(train_digits, (train_digits.shape[0], image_height, image_width, num_channels))
test_data = np.reshape(test_digits, (test_digits.shape[0],image_height, image_width, num_channels))

# re-scale the image data to values between (0.0,1.0]
train_data = train_data.astype('float32') / 255.
test_data = test_data.astype('float32') / 255.

# one-hot encode the labels - we have 10 output classes
# so 3 -> [0 0 0 1 0 0 0 0 0 0], 5 -> [0 0 0 0 0 1 0 0 0 0] & so on
from tensorflow.keras.utils import to_categorical
num_classes = 10
train_labels_cat = to_categorical(train_labels,num_classes)
test_labels_cat = to_categorical(test_labels,num_classes)
train_labels_cat.shape, test_labels_cat.shape

### train test split

In [ ]:
# shuffle the training dataset (5 times!)
for _ in range(5): 
    indexes = np.random.permutation(len(train_data))

train_data = train_data[indexes]
train_labels_cat = train_labels_cat[indexes]

# now set-aside 10% of the train_data/labels as the
# cross-validation sets
val_perc = 0.10
val_count = int(val_perc * len(train_data))

# first pick validation set from train_data/labels
val_data = train_data[:val_count,:]
val_labels_cat = train_labels_cat[:val_count,:]

# leave rest in training set
train_data2 = train_data[val_count:,:]
train_labels_cat2 = train_labels_cat[val_count:,:]

# NOTE: We will train on train_data2/train_labels_cat2 and 
# cross-validate on val_data/val_labels_cat

In [ ]:
train_data2.shape

### Build the neural network


In [ ]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

def build_model():
    model = Sequential()
    # add Convolutional layers
    model.add(Conv2D(filters=32, kernel_size=(3,3), activation='relu', padding='same',
                     input_shape=(image_height, image_width, num_channels)))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Conv2D(filters=64, kernel_size=(3,3), activation='relu', padding='same'))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Conv2D(filters=64, kernel_size=(3,3), activation='relu', padding='same'))
    model.add(MaxPooling2D(pool_size=(2,2)))    
    model.add(Flatten())
    # Densely connected layers
    model.add(Dense(128, activation='relu'))
    # output layer
    model.add(Dense(num_classes, activation='softmax'))
    # compile with adam optimizer & categorical_crossentropy loss function
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

model = build_model()
print(model.summary())

### Train the Model

In [ ]:
results = model.fit(train_data2, train_labels_cat2, 
                    epochs=1, batch_size=64,
                    validation_data=(val_data, val_labels_cat))

### Evaluate Model

In [ ]:
test_loss, test_accuracy = \
  model.evaluate(test_data, test_labels_cat, batch_size=64)
print('Test loss: %.4f accuracy: %.4f' % (test_loss, test_accuracy))

### evaluate its performance

In [ ]:
predictions = model.predict(test_data)
first20_preds = np.argmax(predictions, axis=1)[:9]
first20_true = np.argmax(test_labels_cat,axis=1)[:9]

In [ ]:
plt.figure()
for i, correct in enumerate(first20_preds):
    plt.subplot(3,3,i+1)
    plt.imshow(test_data[i][:,:, 0], cmap='gray', interpolation='none')
    plt.title("Predicted {}, Class {}".format(first20_preds[i], first20_true[i]))
    


# Deployment

In [ ]:
import modelify
from modelify import ModelInference
from modelify.inputs import Image

In [ ]:
my_input = Image(width=28, height=28, channel=1) # images are grayscale

In [ ]:
my_input.add_sample("https://machinelearningmastery.com/wp-content/uploads/2019/02/sample_image.png")

In [ ]:
inference = ModelInference(model=model, framework="KERAS", inputs=my_input)

In [ ]:
inference.test()

## Add postprocess function

In [ ]:
def my_postprocess(outputs):
    import numpy as np
    return np.argmax(outputs, axis=1).tolist() # take the highest probability

In [ ]:
inference.postprocess = my_postprocess

In [ ]:
inference.test()

## Create an API on local

In [ ]:
modelify.run(inference)

-----------------------

# Deploy to Modelify Cloud

In [ ]:
modelify.connect("YOUR_CONNECTION_KEY")

In [ ]:
modelify.deploy(inference, app_uid="YOUR_APP_UID")